In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
import scipy.signal as ss

In [ ]:
raw_data = sio.loadmat('data/raw/m4000Series_m4028.mat')

In [ ]:
lfp = raw_data['m4028_session1'][0][0][0].flatten()
run_speed = raw_data['m4028_session1']['run_speed'][0][0].flatten()
true_ripples = raw_data['m4028_session1']['rippleLocs'][0][0].flatten()

candidate_ripples = sio.loadmat('data/raw/m4028_session1.mat')['rippleIdx'].flatten()
duration = sio.loadmat('data/raw/m4028_session1.mat')['w'].flatten()

In [ ]:
lfp.shape, run_speed.shape, true_ripples.shape

In [ ]:
fs = 2500. 
dt = 1. / fs
t = np.arange(lfp.shape[0]) * dt

In [ ]:
# filtering
b, a = ss.butter(5, Wn=np.array([150, 250]) / (fs/2), btype='bandpass')
fLFP = ss.filtfilt(b, a, lfp)                 

In [ ]:
# hilbert transform
eLFP = abs(ss.hilbert(fLFP))

In [ ]:
# true_ripples, candidate_ripples
ripple_true = np.zeros_like(lfp, dtype=int)
ripple_cand = np.zeros_like(lfp, dtype=int)

ripple_true[true_ripples] = 1
ripple_cand[candidate_ripples] = 1

In [ ]:
df = pd.DataFrame(np.c_[t, lfp, fLFP, eLFP, run_speed, ripple_true, ripple_cand], 
                  columns=['time', 'LFP', 'fLFP', 'eLFP', 'speed', 'ripple_true', 'ripple_cand'])

In [ ]:
df

In [ ]:
plt.figure(figsize=(8, 8))
plt.plot(t, lfp, label='Raw LFP')
plt.plot(t, fLFP, label='filtered LFP')
plt.plot(t, eLFP, label='envelope')


#plt.plot(t, run_speed, label='running speed')

plt.vlines(t[true_ripples], 
           np.zeros(t[true_ripples].size), np.zeros(t[true_ripples].size)+0.05, 
           'k', zorder=10)
plt.vlines(t[candidate_ripples], 
           np.zeros(t[candidate_ripples].size)-0.05, np.zeros(t[candidate_ripples].size), 
           'r', zorder=10)
plt.grid()

#for i in range(true_ripples.shape[0]):
#    plt.scatter(x=t[true_ripples[i]], color='black')
#for i in range(true_ripples.shape[0]):
#    plt.axvline(x=t[true_ripples[i]], color='black')
plt.legend()
plt.show()

In [ ]:
plt.vlines

In [ ]:
# fS, tS, Sxx = signal.spectrogram(lfp[(t>=100) & (t <200)], fs, nfft=256, noverlap=250)

In [ ]:
# plt.figure()
# plt.pcolormesh(tS, fS[fS>=10], np.log(Sxx[fS>=10, ]))
# plt.plot(t[true_ripples], np.zeros(t[true_ripples].size)+200, 'w|')
# plt.xlim(tS.min(), tS.max())
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()